In [56]:
from sqlalchemy.orm import Session
import bcrypt
from cryptography.fernet import Fernet
import base64
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet
import os

In [57]:
def generate_secret_key(user_password: str) -> bytes:
    password_bytes = user_password.encode()
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # Fernet keys are 32 bytes
            salt=salt,
            iterations=100000,
            backend=default_backend())
    key = base64.urlsafe_b64encode(kdf.derive(password_bytes))
    
    return key

In [58]:
generate_secret_key('alex')

b'_-ykqGYzjGvtXcf5K0Y33_UQDk1CtSvNrLNGFBso5rU='

In [59]:
alex = generate_secret_key('alex')
print(alex)

b'Pp7RLi3-I96Wd9DTAqR7EbmwOekTApEilx4PZm-E90U='


In [60]:
alexa = generate_secret_key('alex')
print(alexa)

b'7aHxYB42ZJjQymabNIcaAUQ4IdWxS8RiYJHroPStM3w='


In [61]:
key = Fernet.generate_key()

In [62]:
key

b'exhJVNXLHvjf727hT0kLMy12e-SNeRj8mhTiwXWySxQ='

In [63]:
def get_password_hash(password: str) -> str:
    pwd_bytes = password.encode('utf-8')
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password=pwd_bytes, salt=salt)
    
    return hashed_password

In [64]:
def get_secret_key_hash(secret_key: str) -> str:
    salt = bcrypt.gensalt()
    hashed_password = bcrypt.hashpw(password=secret_key, salt=salt)
    
    return hashed_password

In [65]:
hashed_password = get_secret_key_hash(key)

In [66]:
hashed_password

b'$2b$12$y7MnjmGmx.6Oh7jTps7eX./U3W5.pyzkJx0NAz5BbXh0bV2OZGcRO'

In [67]:
type(hashed_password)

bytes

In [69]:
def generate_static_key(specific_string: str) -> bytes:
    # Convert the specific string to bytes
    specific_bytes = specific_string.encode()

    # Choose a salt (you can generate it randomly or use a fixed value)
    salt = b'MySaltValue'  # Change this to a random value if needed

    # Generate the key using PBKDF2HMAC
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 32 bytes key length
        salt=salt,
        iterations=100000,  # You can adjust the number of iterations as needed
        backend=default_backend()
    )

    # Derive the key
    key = kdf.derive(specific_bytes)

    return key

In [70]:
# Example usage
specific_string = "Your specific string here"
static_key = generate_static_key(specific_string)

# Encode the key as URL-safe base64
url_safe_base64_key = base64.urlsafe_b64encode(static_key)
print(url_safe_base64_key)

b'4Hc5uazbfgX48wcqun0MqhoxLQUUiE338N7HwNh7UPk='


In [71]:
url_safe_base64_key = base64.urlsafe_b64encode(static_key)
print(url_safe_base64_key)

b'4Hc5uazbfgX48wcqun0MqhoxLQUUiE338N7HwNh7UPk='


In [72]:
len(url_safe_base64_key)

44

In [68]:
key_1 = Fernet('password')

ValueError: Fernet key must be 32 url-safe base64-encoded bytes.